In [2]:
from react_agent import GenericReactAgent, ReactAgentConfig, MemoryLevel

# 创建配置
config = ReactAgentConfig(
    output_dir="output",
    memory_level=MemoryLevel.NONE,
    knowledge_file="先验知识.md"
)

# 创建 Agent
agent = GenericReactAgent(config)

# 执行任务
agent.execute_task("say hello")

/tmp/ipykernel_651651/2434408527.py:11: UserWarning: Parameters {'max_tokens'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  agent = GenericReactAgent(config)
2025-07-28 05:34:21,198 - react_agent - WARNING - Failed to apply token counting patch: "ChatOpenAI" object has no field "get_num_tokens"
2025-07-28 05:34:21,201 - react_agent - INFO - Memory disabled - using stateless mode
2025-07-28 05:34:21,204 - react_agent - INFO - GenericReactAgent initialized with memory level: none
2025-07-28 05:34:21,206 - react_agent - INFO - Loaded prior knowledge from: 先验知识.md



> Starting task execution...


> Entering new AgentExecutor chain...


2025-07-28 05:34:21,513 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


Hello! How can I assist you today?

> Finished chain.
> Task completed.
